# はじめに
ビッグデータ、データサイエンスという言葉が日本でも広く知られるようになり、10年近くたったと思います。<BR>
そして昨今DXの掛け声のもと、各社データを使用し新しいビジネスの策定や、新規顧客開拓、業務の効率化など、<BR>
積極的に行おうとしているイメージがありますが、業界によって大きくデータ活用のフェーズが異なると思われます。<BR>
<BR>
今回、Kaggleから、2021年版のデータサイエンティストに向けて行われたサーベイのデータが公開されました。<BR>
回答者を、各業界でデータサイエンティスト(もしくは、同等のポジション)として働かれていると方と仮定し、<BR>
このデータを見ていきたいと思います。<BR>
<BR>

# ハイライト
本データは、2021年9月1日から2021年10月4日のおよそ1か月の間に回答されたサーベイになります。<BR>
<BR>
・全体ではのべ25,973人からの回答があり、そのうち日本人は921人（全体の3.5%）となっております。（Q3）<BR>
・最終学歴は、修士が一番多く、次に学士、博士と続きます。（Q4）<BR>
・職種としては、ソフトウエアエンジニアが圧倒的に多く、データサイエンティストとして働かれている方の2倍ほどいるという <BR> 
   のは、ソフトウエアエンジニアからデータサイエンティストに転職したいという人が多いことを表しているのかもしれません。（Q5）<BR>
・Kaggle参加者のコーディング歴が3年以内では46%であり、5年以内61%となります。<BR>
・データサイエンス職は給与が高いイメージですが1000万円を超すのは全体の12%ほどであり、一方、100万円以下が51%ということから、一部フリーランスの方も入っていると思われます。<BR>
<BR>
これらの他にもいろいろな見方で分析をしていますので、楽しんでいただければと思います。<BR>
<BR>
これを読み終わった後では、ご自身でもこのような切り口で分析したらどうなるだろうという疑問が多々生まれると思います。<BR>
**分析される場合は、以下の　Download CSV file　というリンクからCSVを落として、エクセルのピボットテーブルで見るほうが早いので<BR>
そちらがおすすめです。**<BR>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [ ]:
Survey_2021 = pd.read_csv("../input/kaggle-survey-2021/kaggle_survey_2021_responses.csv")
Survey_JP = Survey_2021[Survey_2021['Q3']=='Japan'].reset_index(drop=True)
Survey_JP['Time from Start to Finish (seconds)']=Survey_JP['Time from Start to Finish (seconds)'].astype('int')

# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "Survey_JP.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# Survey_JP.to_csv("Survey_JP.csv",index=False)

# create a link to download the dataframe
create_download_link(Survey_JP)

# Time from Start to Finish (seconds)
すべての回答が信頼できる回答であるかを見るために、アンケートの回答時間を見てみます。<BR>
<BR>
アンケートは42問あり、すべて選択形式です。極端に短い時間で答えていた場合、途中でアンケートを辞めたか、<BR>
あまり真面目に行われていない可能性があるために、それらの回答は省きます。<BR>
<BR>
ヒストグラムからわかるように、一部の人が大きく時間をかけて回答していることがわかります。<BR>
最短回答時間が125秒、最長回答時間が2210790秒（約26日）となっております。<BR>
最長回答時間が長いのは一時保存をして、時間が空いてからの提出だったと想像します。<BR>
回答時間が短かったアンケート回答者のデータが信頼できるものか見てみましょう。

In [ ]:
df=Survey_JP['Time from Start to Finish (seconds)'].reset_index(drop=False)
df.columns=['idx','ResponseTime']

plt.figure(figsize=(15,8)) 

sns.distplot(df['ResponseTime'], bins=20, label='data',kde=False,rug=False)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Time",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.title("Survey response time",fontsize=14)
plt.show()

以下のBarplotからも、3分以内に回答が終わっている方も相当数おり、時間が短いから途中で離脱したとの仮定は正しくなさそうなことがわかります。<BR>
<BR>
グラフを作ってから気づいたのですが、Kaggle_survey_2021_methodologyの中に以下のような説明がありました。これで2分以内にアンケートが終わった人が<BR>
いなかった理由もわかりました。最初に全部確認しないといけないですね。<BR>
<BR>
To ensure response quality, we excluded respondents that were flagged by our survey system as　“Spam” or "Duplicate. <BR>
**We also dropped responses from respondents that spent less than 2　minutes completing the survey**, as well as responses <BR>
from respondents that selected fewer than 15 answer choices in total.<BR>
<BR>
ここからアンケートの分析に入ります。<BR>

In [ ]:
def time_bin(x):
    if x<120:
        return '0 ~ 2 mintues'
    elif x<180:
        return '2 ~ 3 mintues'
    elif x<300:
        return '3 ~ 5 mintues'
    elif x<600:
        return '5 ~ 10 mintues'

    elif x<900:
        return '10 ~ 15 mintues'
    elif x<1800:
        return '15 ~ 30 mintues'
    elif x<3600:
        return '30 ~ 60 mintues'
    else:
        return '60 ~ mintues'

Survey_JP['TimeCategory']=Survey_JP['Time from Start to Finish (seconds)'].apply(time_bin)

df=Survey_JP['TimeCategory'].sort_values(ascending=False).reset_index()
df.columns = ['Time','Category']

Order_list = ['0 ~ 2 mintues','2 ~ 3 mintues','3 ~ 5 mintues','5 ~ 10 mintues','10 ~ 15 mintues','15 ~ 30 mintues','30 ~ 60 mintues','60 ~ mintues']

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Category',y='Time',data = df, alpha=0.7,order=Order_list)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Time",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.title("Survey response time",fontsize=14)
plt.show()

# Q1: What is your age (#years)?
最初の質問は、Kaggle参加者の年齢です。<BR>
ボリュームゾーンは、25歳から34歳辺りのようですが、以外にも50歳以上の方も相当数参加されているようです。

In [ ]:
df = Survey_JP['Q1'].value_counts().reset_index()
df.columns = ['Age','Count']
df.sort_values(by='Age',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Age', y = 'Count', data = df.sort_values(by='Age',ascending=True), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Age",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

# Q2 What is your gender?
性別は想像の通り、男性が多く8割以上を占めています。<BR>
Man、Woman以外の表記があるのが今の時代を表していると思います。<BR>

In [ ]:
df = Survey_JP['Q2'].value_counts(True).reset_index()
df.columns = ['Sex','Count']
df.sort_values(by='Sex',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Sex', y = 'Count', data = df.sort_values(by='Sex',ascending=True), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Sex",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

# Q3 In which country do you currently reside?
世界と日本の立ち位置を比較するために、こちらだけ日本以外のデータも使用しています。<BR>
<BR>
KaggleのDiscussionなどを見ていると、やはりインドの方が多いのを感じます。日本は以外にもアメリカに次いで3位のようです。<BR>
中国より参加者が多いのは意外ですが、これはアンケートに回答した中国の方が少なかったからかもしれません。<BR>
その他にブラジル、ロシアなどが続きますが、Grand MasterのGilberto Titericzさんはブラジル人ですし、<BR>
Courseraの「How to Win a Data Science Competition: Learn from Top Kagglers」はロシアのTop Kagglerの<BR>
講義でしたので、なんとなく皆さん雰囲気的に納得されるかと思います。<BR>
<BR>
**ナイジェリアがこんなに多いのが謎です。。。ドメイン知識がある方はコメント欄にコメントをしていただければ。<BR>**

In [ ]:
df = Survey_2021['Q3'].value_counts().sort_values(ascending=False).reset_index().head(10)
df.columns = ['Country','Count']
df.sort_values(by='Country',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Country', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Country",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

# Q4 : What is the highest level of formal education that you have attained or plan to attain within the next 2 years?
Kaggle参加者は理系出身の方が多いと想像されますが、そのせいか、最終学歴が修士であるかたが一番多く、次に学士が続きます。<BR>
博士を持っているかたは、もしかしたらML Researcherなどの方かもしれません。<BR>

In [ ]:
df = Survey_JP['Q4'].value_counts().sort_values(ascending=False).reset_index().head(7)
df.columns = ['Education','Count']
df.sort_values(by='Education',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Education', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Education",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

# Q5 : Select the title most similar to your current role (or most recent title if retired)
現在の職業は、以外にもデータサイエンティストではなく、Software Engineerの方が1番多いようです。<BR>
これはResearch ScientistとData Scientistを合計した程度の数字になります。<BR>
次に多いのは学生であり、これは大学でも機械学習の授業が取り入れ始められてきたからだと思われます。<BR>
<BR>
うちは家族でテック系Pod CastのRebuiled FMを聞くのですが、この前行われたindoor LocationでCompetitions<BR>
MasterになられたhigeponさんもSoftware Engineerですね。<BR>

In [ ]:
df = Survey_JP['Q5'].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

職業が出てきたので、Q4の博士の方の職業を見てみましょう。<BR>
以下が、博士ホルダーの方々で、やはりResearch Scientistが多いようです。<BR>

In [ ]:
df = Survey_JP[Survey_JP['Q4']=='Doctoral degree']['Q5'].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

博士を取られる方はまだまだ日本では少数だと思いますので、修士の就職先も見てみましょう。<BR>
<BR>
博士と修士では就いているポジションが違いますね。<BR>

In [ ]:
df = Survey_JP[Survey_JP['Q4']=='Master’s degree']['Q5'].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

# Q6 : For how many years have you been writing code and/or programming?
Kaggle参加者のプログラミング歴は、Software Engineerの方が多いこともあり、長いのかなと想像していましたが、<BR>
ボリュームゾーンは3年以下のようです。これは直感と会わないので、職業と組み合わせて見てみましょう。<BR>

In [ ]:
Order_list = ['I have never written code','< 1 years','1-3 years','3-5 years','5-10 years','10-20 years','20+ years']

df = Survey_JP['Q6'].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Coding experiences','Count']
df.sort_values(by='Coding experiences',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Coding experiences', y = 'Count', data = df.sort_values(by='Count',ascending=False),order = Order_list, alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.show()

# Q5 & Q6 Current role & Programming experience
以下は、職業別にプログラミング歴をBarplotで表したものです。<BR>
やはり、3年未満までは学生が多く、Software Engineerの方はプログラミング歴が長いですね。<BR>

In [ ]:
Order_list = ['I have never written code','< 1 years','1-3 years','3-5 years','5-10 years','10-20 years','20+ years']

Survey_Q5_Q6=Survey_JP.groupby(['Q5','Q6'])['Q1'].count().reset_index(drop=False)
df = Survey_Q5_Q6[(Survey_Q5_Q6['Q5']=='Software Engineer')|(Survey_Q5_Q6['Q5']=='Student')|(Survey_Q5_Q6['Q5']=='Other')|
                  (Survey_Q5_Q6['Q5']=='Research Scientist')|(Survey_Q5_Q6['Q5']=='Data Scientist')|(Survey_Q5_Q6['Q5']=='Data Scientist''Business Analyst')]

 
plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Q6', y = 'Q1', hue = 'Q5', data = df, order = Order_list,alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Coding",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.ylim([0,100])
plt.title("Current role & Programming experience",fontsize=14)
plt.show()

# Q7. What programming languages do you use on a regular basis?
日頃使うプログラミング言語についてですが、こちらはやはり、Pythonが圧倒的に1位ですね。意外とRが他の言語に比べて少ないです。<BR>
データサイエンティストと言っても、機械学習でモデルを作成する人もいれば、ABテストの設計や実施を行う人など、幅広くいると思います。<BR>
目的によって、使う言語が変わると思いますが、Rの少なさは意外でした。<BR>
VBAは選択肢にも入らない時代です。<BR>

In [ ]:
#  Q7_Part_1 : What programming languages do you use on a regular basis? (Select all that apply) - Selected Choice - Python
Cols = ['Q7_Part_1','Q7_Part_2','Q7_Part_3','Q7_Part_4','Q7_Part_5','Q7_Part_6','Q7_Part_7','Q7_Part_8','Q7_Part_9','Q7_Part_10','Q7_Part_11','Q7_Part_12','Q7_OTHER']
Programing = ['Python','R','SQL','C','C++','Java','Javascript','Julia','Swift','Bash','MATLAB','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['Progaming','Count']


plt.figure(figsize=(15,8)) 
sns.barplot(x ='Progaming' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,1000))
plt.title('What programming languages do you use on a regular basis?',fontsize=14)
plt.show()

# Q8 : What programming language would you recommend an aspiring data scientist to learn first?
初学者が学ぶべき言語は圧倒的にPythonなようです。これは、文法的に読みやすい言語であることに加え、本屋さんにもいい本が<BR>
日本語でたくさん出てきているからだと思われます。<BR>
<BR>
初学者に、CやC++をおすすめしている方は、かなりつよつよな方かもしれません。

In [ ]:
df=Survey_JP['Q8'].value_counts().reset_index()
df.columns = ['Program','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Program', data = df.sort_values(by='Count',ascending=False))

# plt.xticks(rotation=270)
# plt.ylim((0,1000))
plt.yticks(fontsize=15)
plt.show()

# Q9.Which of the following integrated development environments (IDE's) do you use on a regular basis? 
IDEは、JupyterNotebookとVSCodeがやはり人気みたいですね。<BR>
ただこの2つで、使っている方の職業が違うような気がします。<BR>

In [ ]:
#  Which of the following integrated development environments (IDE's) do you use on a regular basis? 
Cols = ['Q9_Part_1','Q9_Part_2','Q9_Part_3','Q9_Part_4','Q9_Part_5','Q9_Part_6','Q9_Part_7','Q9_Part_8','Q9_Part_9','Q9_Part_10','Q9_Part_11','Q9_Part_12','Q9_OTHER']
Programing = ['Jupyter','RStudio','Visual Studio','Visual Studio Code (VSCode)','PyCharm','Spyder','Notepad++','Sublime Text','Vim / Emacs','MATLAB','Jupyter Notebook','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['Progaming','Count']


plt.figure(figsize=(15,8)) 
sns.barplot(x ='Progaming' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,600))
plt.title("Which of the following integrated development environments (IDE's) do you use on a regular basis?",fontsize=14)
plt.show()

以下はそれぞれ、JupyterNotebookをメインで使っているユーザー、VSCodeをメインで使っているユーザーのBarplotですが、<BR>
Softwear engineerの方も相応にJupyterNotebookを使われているようです。<BR>

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q9_Part_11']==' Jupyter Notebook')].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)


# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.title("Jupyter Notebook User",fontsize=12)
plt.show()

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q9_Part_4']==' Visual Studio Code (VSCode) ')].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)


# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.title("VSCode User",fontsize=12)
plt.show()

# Q11 : What type of computing platform do you use most often for your data science projects?
現在は、Google Colaboratoryなどクラウドベースでのインフラが整ってきているため、メインはlaptopで分析をし、<BR>
必要に応じてクラウドを使用するという感じなのでしょうか？<BR>
<BR>
本当にここ10年で分析環境が変わった気がします。<BR>

In [ ]:
#Q11 : What type of computing platform do you use most often for your data science projects? - Selected Choice

df=Survey_JP['Q11'].value_counts().reset_index()
df.columns = ['Platform','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Platform', data = df.sort_values(by='Count',ascending=False), orient = 'h')

# plt.xticks(rotation=270)
# plt.ylim((0,1000))
plt.yticks(fontsize=15)
plt.show()

# Q13 : Approximately how many times have you used a TPU (tensor processing unit)?
TPUを何回くらいつかったことがあるかという質問ですが、ほとんどの人はTPUをつかったことはなく、<BR>
一部の人が大量に使っているのが見えます。<BR>

In [ ]:
df=Survey_JP['Q13'].value_counts().reset_index()
df.columns = ['Time','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Time', data = df.sort_values(by='Count',ascending=False), orient = 'h')

# plt.xticks(rotation=270)
# plt.ylim((0,1000))
plt.yticks(fontsize=15)
plt.show()

# Q15 : For how many years have you used machine learning methods?
機械学習は、歴史こそある物の、一般的な人が使うようになったのはここ5～10年だと思いますが、<BR>
以外にも1年以内の方が1番多いようです。コロナ禍を機会に勉強した人が多いということでしょうか？<BR>

In [ ]:
# Q15 : For how many years have you used machine learning methods?

df=Survey_JP['Q15'].value_counts().reset_index()
df.columns = ['Platform','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Platform', data = df.sort_values(by='Count',ascending=False), orient = 'h')

# plt.xticks(rotation=270)
# plt.ylim((0,1000))
plt.yticks(fontsize=15)
plt.show()

こちらも職業と合わせて見てみましょう。<BR>
学生だけでなく、Software engineerの方も、ここ2,3年の間で機械学習の勉強を始めたという方が多いようですね。<BR>
<BR>

In [ ]:
Order_list = ['I do not use machine learning methods','Under 1 year','1-2 years','2-3 years','3-4 years','4-5 years','5-10 years','10-20 years','20 or more years']


Survey_gpby=Survey_JP.groupby(['Q5','Q15'])['Q1'].count().reset_index(drop=False)
df = Survey_gpby[(Survey_gpby['Q5']=='Software Engineer')|(Survey_gpby['Q5']=='Student')|(Survey_gpby['Q5']=='Other')|
                   (Survey_gpby['Q5']=='Research Scientist')|(Survey_gpby['Q5']=='Data Scientist')|(Survey_gpby['Q5']=='Data Scientist''Business Analyst')]

 
plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Q15', y = 'Q1', hue = 'Q5',data = df, order = Order_list, alpha=0.7) 
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Coding Experience",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.ylim([0,100])
plt.title("Current role & Programming experience",fontsize=14)
plt.show()

# Q16: Which of the following machine learning frameworks do you use on a regular basis?
日頃使うフレームワークは、想像通りScikit-learnが多いですが、その後が、TensorFlow,PyTorchと来ているのを見ると<BR>
やはりDeepLearningも必須になりつつあるということなのでしょうか？<BR>
<BR>
Caret、H2O辺りは、AutoML系だと思いますが、あまりまだ浸透していないのかもしれません。<BR>
尚、こちらは複数選択可能な問題です。

In [ ]:
#  Which of the following integrated development environments (IDE's) do you use on a regular basis? 
Cols = ['Q16_Part_1','Q16_Part_2','Q16_Part_3','Q16_Part_4','Q16_Part_5','Q16_Part_6','Q16_Part_7','Q16_Part_8','Q16_Part_9','Q16_Part_10',
        'Q16_Part_11','Q16_Part_12','Q16_Part_13','Q16_Part_14','Q16_Part_15','Q16_Part_16','Q16_Part_17','Q16_OTHER']

Programing = ['Scikit-learn','TensorFlow','Keras','PyTorch','Fast.ai','MXNet','Xgboost','LightGBM','CatBoost','Prophet','H2O 3','Caret',
             'Tidymodels','JAX','PyTorch Lightning','Huggingface','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['Frameworks','Count']


plt.figure(figsize=(15,8)) 
sns.barplot(x ='Frameworks' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,600))
plt.title("Which of the following machine learning frameworks do you use on a regular basis?",fontsize=14)
plt.show()

DeepLearningのFrameworkを使う方の職業を知りたいので、こちらも職業別に分けてみましょう。

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q16_Part_1']=='  Scikit-learn ')].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.title("Scikit-learn",fontsize=12)
plt.show()

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q16_Part_2']=='  TensorFlow ')].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.title("TensorFlow",fontsize=12)
plt.show()

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q16_Part_4']==' PyTorch ')].value_counts().sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Count",fontsize=12)
plt.title("PyTorch",fontsize=12)
plt.show()

# Q17: Which of the following ML algorithms do you use on a regular basis?
よく使われるライブラリーがScikit-learnであることからも、上位はLogisticRegression、DecisionTreeなどが来ています。解釈可能なモデルがビジネスシーンで<BR>
求められているのも一因でしょうか。<BR>
<BR>
こちらも職業別にみてみましょう。

In [ ]:
Cols = ['Q17_Part_1','Q17_Part_2','Q17_Part_3','Q17_Part_4','Q17_Part_5','Q17_Part_6','Q17_Part_7',
        'Q17_Part_8','Q17_Part_9','Q17_Part_10','Q17_Part_11','Q17_OTHER']

Programing = ['LinearorLogisticRegression','DecisionTreesorRandomForests','GradientBoostingMachines(xgboost,lightgbm,etc)',
              'BayesianApproaches','EvolutionaryApproaches','DenseNeuralNetworks(MLPs,etc)','ConvolutionalNeuralNetworks',
              'GenerativeAdversarialNetworks','RecurrentNeuralNetworks','TransformerNetworks(BERT,gpt','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['Algorithms','Count']


plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Algorithms', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.yticks(fontsize=15)
plt.show()

以下３つのグラフは、職業×機械学習アルゴリズムです。<BR>

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q17_Part_3']=='Gradient Boosting Machines (xgboost, lightgbm, etc)')].value_counts(True).sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Rate",fontsize=12)
plt.title("Gradient Boosting Machines (xgboost, lightgbm, etc)",fontsize=12)
plt.show()

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q17_Part_7']=='Convolutional Neural Networks')].value_counts(True).sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Rate",fontsize=12)
plt.title("Convolutional Neural Networks",fontsize=12)
plt.show()

In [ ]:
df = Survey_JP['Q5'][(Survey_JP['Q17_Part_10']=='Transformer Networks (BERT, gpt-3, etc)')].value_counts(True).sort_values(ascending=False).reset_index()
df.columns = ['Profession','Count']
df.sort_values(by='Profession',ascending=False)

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'Profession', y = 'Count', data = df.sort_values(by='Count',ascending=False), alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Profession",fontsize=12)
plt.ylabel("Rate",fontsize=12)
plt.title("Transformer Networks (BERT, gpt-3, etc)",fontsize=12)
plt.show()

# Q20 : In what industry is your current employer/contract (or your most recent employer if retired)?
日本における各業界ごとのデータサイエンスの広がりの現状がこちらで見えると思います。<BR>
1番活用されているのはIT業界ですが、次に製造業、教育関係、製薬業界と続きます。ここら辺の感覚は個人的な感覚とはあっており<BR>
金融に関しては生損保が銀行、証券業界よりも進んでいる実感があります。<BR>
アンケートに答えている方が偏っている可能性も高いのですが、大まかな現状が現れていると思われます。

In [ ]:
# Q20 : In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice

df=Survey_JP['Q20'].value_counts().reset_index()
df.columns = ['Profession','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Profession', data = df.sort_values(by='Count',ascending=False), orient = 'h')

plt.yticks(fontsize=15)
plt.show()

# Q21 : What is the size of the company where you are employed?
会社の規模は、1000以上の会社が大半のようです。<BR>
規模が0-49人のスタートアップと思われるものが、全体の17.6%（120人）というのも面白いデータだと思います。<BR>

In [ ]:
# Q21 : What is the size of the company where you are employed?
df=Survey_JP['Q21'].value_counts(True).reset_index()
df.columns = ['Profession','Rate']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Rate' , y = 'Profession', data = df.sort_values(by='Rate',ascending=False), orient = 'h')

plt.yticks(fontsize=15)
plt.show()

# Q22 : Approximately how many individuals are responsible for data science workloads at your place of business?
Q21の中で、回答者の大半が大企業所属ということはわかりました。これを踏まえて、データサイエンスの責任者がどのくらいいるかを見てみましょう。<BR>
<BR>
大多数で責任者が2名以下の企業が多いようですが、次に20名以上となっています。<BR>
**20名以上となっている業界（Q20）を見てみるとManufacturing/Fabricationが45、Computers/Technologyが28となっています。<BR>**

In [ ]:
#Q22 : Approximately how many individuals are responsible for data science workloads at your place of business?
df=Survey_JP['Q22'].value_counts().reset_index()
df.columns = ['Member','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Member', data = df.sort_values(by='Count',ascending=False), orient = 'h')

plt.title("Q22 : Approximately how many individuals are responsible for data science workloads at your place of business?")
plt.yticks(fontsize=15)
plt.show()

# Q23 : Does your current employer incorporate machine learning methods into their business?

雇用主は、機械学習の手法をビジネスに取り入れていますか？という問いですが、まだまだ機械学習使用したモデルを製品、サービスに落とし込むのは<BR>
一般的ではなく、最近機械学習を取り入れ模索しているというところでしょうか。<BR>

In [ ]:
# Q23 : Does your current employer incorporate machine learning methods into their business?
df=Survey_JP['Q23'].value_counts().reset_index()
df.columns = ['Profession','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Profession', data = df.sort_values(by='Count',ascending=False), orient = 'h')

plt.yticks(fontsize=15)
plt.title("Q23 : Does your current employer incorporate machine learning methods into their business?")
plt.show()

MLを現在よく使っていると行っているのが、どの程度の規模の会社に所属している人かを見てみましょう。<BR>
<BR>
以下はQ23で2年以上前から製品、サービスに取り込んでいると回答した人が所属する業種になりますが、やはりIT、製造業が圧倒的に多く、<BR>
他の業種は大きく遅れているように感じられます。<BR>

In [ ]:
# Q23 : Does your current employer incorporate machine learning methods into their business?
df=Survey_JP['Q20'][Survey_JP['Q23']=='We have well established ML methods (i.e., models in production for more than 2 years)'].value_counts().reset_index()
df.columns = ['Profession','Count']

plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Profession', data = df.sort_values(by='Count',ascending=False), orient = 'h')

plt.yticks(fontsize=15)
plt.title("We have well established ML methods (i.e., models in production for more than 2 years)")
plt.show()

# Q24: Select any activities that make up an important part of your role at work

重要な部分を占める活動を選んでください。という問いに対しては、データの本質を理解しビジネスの意思決定に生かすという<BR>
データサイエンス、データアナリストに求められるべきところが求められている感じがします。<BR>
<BR>
但し、2番目が、特段求められている役割が選択肢にないということで、データドリブンな意思決定が浸透してきているのは<BR>
一部の企業だけだと思われます。<BR>
<BR>
**ちなみにビジネスの意思決定に貢献すると答えた人のポジション（Q5）は、データサイエンティスト56、Softwear Engineer50、データアナリスト36となっています。<BR>**

In [ ]:
Cols = ['Q24_Part_1','Q24_Part_2','Q24_Part_3','Q24_Part_4','Q24_Part_5','Q24_Part_6','Q24_Part_7','Q24_OTHER']

Programing = ['Analyze and understand data to influence product or business decisions',
              'Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data',
              ' Build prototypes to explore applying machine learning to new areas',
              'Build and/or run a machine learning service that operationally improves my product or workflows',
              'Experimentation and iteration to improve existing ML models',
              'Do research that advances the state of the art of machine learning',
              'None of these activities are an important part of my role at work','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['Task','Count']


plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Task', data = df.sort_values(by='Count',ascending=False), orient = 'h')

plt.xticks(rotation=270)
plt.yticks(fontsize=15)
plt.show()

# Q25 : What is your current yearly compensation (approximate $USD)?
比較的給与が高いと思われるデータサイエンティストですが、まずはKaggleに参加されているかたの給与の分布を見てみましょう。<BR>
100万円以下はフリーランスで副業で行われている方でしょうか。**(こちらのデータにはStudentは含まれておりません)**<BR>
<BR>
100万円以上を対象にするとボリュームゾーンは400万円から700万円のようです。<BR>
<BR>
次にQ5の職業ごとに給与の分布を載せました。<BR>

In [ ]:
Order_list=['$0-999','1,000-1,999', '2,000-2,999', '3,000-3,999','4,000-4,999','5,000-7,499', '7,500-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999',  '30,000-39,999',
            '40,000-49,999','50,000-59,999', '60,000-69,999','70,000-79,999','80,000-89,999','90,000-99,999','100,000-124,999', '125,000-149,999','150,000-199,999', '200,000-249,999',
            '250,000-299,999','300,000-499,999', '$500,000-999,999','>$1,000,000']
         
plt.figure(figsize=(18,8)) 

df = Survey_JP['Q25'].value_counts().reset_index()
df.columns = ['Salary','Count']
sns.barplot(x = 'Salary', y = 'Count', data = df, order = Order_list,alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Salary")
plt.ylabel("Count")
plt.title("What is your current yearly compensation (approximate $USD)?")
plt.show()

こちらも職業別に分けていきます。<BR>
Researchの方のほうが、Data Scientistの方よりも給与が良いイメージがあったのですが、必ずしもそうではないようです。<BR>
非常に面白いと思ったのがQ5でOtherと答えている人たちの一部のかたが高所得であるということです。<BR>
これは、コンサル業界の方ではないかと想像しています。<BR>

In [ ]:
Order_list=['$0-999','1,000-1,999', '2,000-2,999', '3,000-3,999','4,000-4,999','5,000-7,499', '7,500-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999',  '30,000-39,999',
            '40,000-49,999','50,000-59,999', '60,000-69,999','70,000-79,999','80,000-89,999','90,000-99,999','100,000-124,999', '125,000-149,999','150,000-199,999', '200,000-249,999',
            '250,000-299,999','300,000-499,999', '$500,000-999,999','>$1,000,000']

plt.figure(figsize=(18,8)) 

df = Survey_JP['Q25'][Survey_JP['Q5']=='Software Engineer'].value_counts().reset_index()
df.columns = ['Salary','Count']
sns.barplot(x = 'Salary', y = 'Count', data = df, order = Order_list,alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Salary")
plt.ylabel("Count")
plt.title("Profession: Software Engineer")
plt.show()

In [ ]:
Order_list=['$0-999','1,000-1,999', '2,000-2,999', '3,000-3,999','4,000-4,999','5,000-7,499', '7,500-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999',  '30,000-39,999',
            '40,000-49,999','50,000-59,999', '60,000-69,999','70,000-79,999','80,000-89,999','90,000-99,999','100,000-124,999', '125,000-149,999','150,000-199,999', '200,000-249,999',
            '250,000-299,999','300,000-499,999', '$500,000-999,999','>$1,000,000']

plt.figure(figsize=(18,8)) 

df = Survey_JP['Q25'][Survey_JP['Q5']=='Data Scientist'].value_counts().reset_index()
df.columns = ['Salary','Count']
sns.barplot(x = 'Salary', y = 'Count', data = df, order = Order_list,alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Salary")
plt.ylabel("Count")
plt.title("Profession: Data Scientist")
plt.show()

In [ ]:
Order_list=['$0-999','1,000-1,999', '2,000-2,999', '3,000-3,999','4,000-4,999','5,000-7,499', '7,500-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999',  '30,000-39,999',
            '40,000-49,999','50,000-59,999', '60,000-69,999','70,000-79,999','80,000-89,999','90,000-99,999','100,000-124,999', '125,000-149,999','150,000-199,999', '200,000-249,999',
            '250,000-299,999','300,000-499,999', '$500,000-999,999','>$1,000,000']

plt.figure(figsize=(18,8)) 

df = Survey_JP['Q25'][Survey_JP['Q5']=='Research Scientist'].value_counts().reset_index()
df.columns = ['Salary','Count']
sns.barplot(x = 'Salary', y = 'Count', data = df, order = Order_list,alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Salary")
plt.ylabel("Count")
plt.title("Profession: Research Scientist")
plt.show()

In [ ]:
Order_list=['$0-999','1,000-1,999', '2,000-2,999', '3,000-3,999','4,000-4,999','5,000-7,499', '7,500-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999',  '30,000-39,999',
            '40,000-49,999','50,000-59,999', '60,000-69,999','70,000-79,999','80,000-89,999','90,000-99,999','100,000-124,999', '125,000-149,999','150,000-199,999', '200,000-249,999',
            '250,000-299,999','300,000-499,999', '$500,000-999,999','>$1,000,000']

plt.figure(figsize=(18,8)) 

df = Survey_JP['Q25'][Survey_JP['Q5']=='Machine Learning Engineer'].value_counts().reset_index()
df.columns = ['Salary','Count']
sns.barplot(x = 'Salary', y = 'Count', data = df, order = Order_list,alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Salary")
plt.ylabel("Count")
plt.title("Profession: Machine Learning Engineer")
plt.show()

In [ ]:
Order_list=['$0-999','1,000-1,999', '2,000-2,999', '3,000-3,999','4,000-4,999','5,000-7,499', '7,500-9,999', '10,000-14,999', '15,000-19,999', '20,000-24,999', '25,000-29,999',  '30,000-39,999',
            '40,000-49,999','50,000-59,999', '60,000-69,999','70,000-79,999','80,000-89,999','90,000-99,999','100,000-124,999', '125,000-149,999','150,000-199,999', '200,000-249,999',
            '250,000-299,999','300,000-499,999', '$500,000-999,999','>$1,000,000']

plt.figure(figsize=(18,8)) 

df = Survey_JP['Q25'][Survey_JP['Q5']=='Other'].value_counts().reset_index()
df.columns = ['Salary','Count']
sns.barplot(x = 'Salary', y = 'Count', data = df, order = Order_list,alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Salary")
plt.ylabel("Count")
plt.title("Profession: Other")
plt.show()

#  Q26 : Approximately how much money have you (or your team) spent on machine learning and/or cloud computing services at home (or at work) in the past 5 years (approximate $USD)?
過去5年でコンピューティングリソースのためにいくら使ったかという質問ですが、多くの方は10万円以内（パソコンを1台買うくらい）のようですが、<BR>
一部の方はサーバーを使いかなり計算をさせているということを表しているのかもしれません。<BR>

In [ ]:
#Q26 : Approximately how much money have you (or your team) spent on machine learning and/or cloud computing services at home (or at work) in the past 5 years (approximate $USD)?
df = Survey_JP['Q26'].value_counts(True).reset_index()
df.columns = ['Money','Count']

Order_list=['$0 ($USD)','$1-$99','$100-$999','$1000-$9,999','$10,000-$99,999','$100,000 or more ($USD)']


plt.figure(figsize=(18,8)) 

sns.barplot(x = 'Money', y = 'Count', data = df, order = Order_list, alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("spent on machine learning in the past 5 years ($USD)")
plt.ylabel("Rate")
plt.title("Q26 : Approximately how much money have you (or your team) spent on machine learning and/or cloud computing services at home (or at work) in the past 5 years (approximate $USD)?")
plt.show()

# Q28 : Of the cloud platforms that you are familiar with, which has the best developer experience (most enjoyable to use)? 

In [ ]:
df = Survey_JP['Q28'].value_counts(True).reset_index()
df.columns = ['Platform','Count']

plt.figure(figsize=(18,8)) 

sns.barplot(x = 'Platform', y = 'Count', data = df, alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Platform")
plt.ylabel("Count")
plt.show()

# Q32: Which of the following big data products (relational databases, data warehouses, data lakes, or similar) do you hope to become more familiar with in the next 2 years?

In [ ]:
Cols = ['Q32_B_Part_1','Q32_B_Part_2','Q32_B_Part_3','Q32_B_Part_4','Q32_B_Part_5','Q32_B_Part_6','Q32_B_Part_7',
        'Q32_B_Part_8','Q32_B_Part_9','Q32_B_Part_10','Q32_B_Part_11','Q32_B_Part_12','Q32_B_Part_13','Q32_B_Part_14',
        'Q32_B_Part_15','Q32_B_Part_16','Q32_B_Part_17','Q32_B_Part_18','Q32_B_Part_19','Q32_B_Part_20','Q32_B_OTHER']

Programing = ['MySQL','PostgreSQL','SQLite','Oracle Database','MongoDB','Snowflake','IBM Db2','Microsoft SQL Server','Microsoft Azure SQL Database',
              'Microsoft Azure Cosmos DB',' Amazon Redshift','Amazon Aurora','Amazon DynamoDB','Amazon RDS','Google Cloud BigQuery','Google Cloud SQL',
              'Google Cloud Firestore','Google Cloud BigTable','Google Cloud Spanner','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['Product','Count']


plt.figure(figsize=(15,8)) 
sns.barplot(x ='Count' , y = 'Product', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.yticks(fontsize=15)
plt.show()

# Q33 : Which of the following big data products (relational database, data warehouse, data lake, or similar) do you use most often? - Selected Choice

In [ ]:
df = Survey_JP['Q33'].value_counts(True).reset_index()
df.columns = ['Product','Count']

plt.figure(figsize=(18,8)) 

sns.barplot(x = 'Product', y = 'Count', data = df, alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("Product")
plt.ylabel("Rate")
plt.show()

# Q34: Which of the following business intelligence tools do you hope to become more familiar with in the next 2 years?

In [ ]:
Cols = ['Q34_B_Part_1','Q34_B_Part_2','Q34_B_Part_3','Q34_B_Part_4','Q34_B_Part_5','Q34_B_Part_6','Q34_B_Part_7','Q34_B_Part_8','Q34_B_Part_9','Q34_B_Part_10',
        'Q34_B_Part_11','Q34_B_Part_12','Q34_B_Part_13','Q34_B_Part_14','Q34_B_Part_15','Q34_B_Part_16','Q34_B_OTHER']

Programing = [' Microsoft Power BI',' Amazon QuickSight',' Google Data Studio',' Looker',' Tableau',' Salesforce',
              ' Tableau CRM',' Qlik',' Domo',' TIBCO Spotfire',' Alteryx ',' Sisense ',' SAP Analytics Cloud ',' Microsoft Azure Synapse ',' Thoughtspot ',' None',' Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['BI tools','Count']

plt.figure(figsize=(18,8)) 
sns.barplot(x ='BI tools' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,200))
plt.show()

# Q35 : Which of the following business intelligence tools do you use most often?
回答者の中では、BIツールはTableauが1位で、次にMS PowerBIとなっていますね。

In [ ]:
df = Survey_JP['Q35'].value_counts(True).reset_index()
df.columns = ['business intelligence','Count']

plt.figure(figsize=(18,8)) 

sns.barplot(x = 'business intelligence', y = 'Count', data = df, alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("business intelligence")
plt.ylabel("Rate")
plt.show()

# Q37 : Which specific automated machine learning tools (or partial AutoML tools) do you hope to become more familiar with in the next 2 years? 
まだまだAutoMLは浸透していない印象がありまっすが、Googleについで、DataRobotが入ってきています。<BR>
**DataRobotを入れている業界（Q20）と合わせてみると、製造業が3、Otherが3ということです。<BR>**

In [ ]:
Cols = ['Q37_A_Part_1','Q37_A_Part_2','Q37_A_Part_3','Q37_A_Part_4','Q37_A_Part_5','Q37_A_Part_6','Q37_A_Part_7','Q37_A_OTHER']
Programing = ['Google Cloud AutoML','H2O Driverless AI','Databricks AutoML','DataRobot AutoML','Amazon Sagemaker Autopilot','Azure Automated Machine Learning','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['AutoML tools','Count']


plt.figure(figsize=(18,8)) 
sns.barplot(x ='AutoML tools' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,80))
plt.show()

# Q39: Where do you publicly share your data analysis or machine learning applications?
データ分析や機械学習のアプリケーションをどこで公開しているかですが、Kaggleの方にアンケートを取っているので、<BR>
Kaggleが1位で、２位はGitHubです。これも想像の通りではないでしょうか。

In [ ]:
Cols = ['Q39_Part_1','Q39_Part_2','Q39_Part_3','Q39_Part_4','Q39_Part_5','Q39_Part_6','Q39_Part_7','Q39_Part_8','Q39_Part_9','Q39_OTHER']
Programing = ['Plotly Dash','Streamlit','NBViewer','GitHub','Personal blog','Kaggle','Colab','Shiny','I do not share my work publicly','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['AutoML tools','Count']


plt.figure(figsize=(18,8)) 
sns.barplot(x ='AutoML tools' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,200))
plt.show()

# Q40 : On which platforms have you begun or completed data science courses?
機械学習系の教材も最近では多く、フリーで受けられる質のいいものも多いため、勉強したい人には時代だと思います。<BR>
それもあって、インドやナイジェリアの方が勉強し、Kaggleに参加されているのでしょう。<BR>
こちらも、Kaggle、Udemy、Courseraの順で、ある程度想像通りです。<BR>

In [ ]:
Cols = ['Q40_Part_1','Q40_Part_2','Q40_Part_3','Q40_Part_4','Q40_Part_5','Q40_Part_6','Q40_Part_7','Q40_Part_8','Q40_Part_9','Q40_Part_10','Q40_Part_11','Q40_OTHER']
Programing = [' Coursera','edX','Kaggle Learn Courses','DataCamp','Fast.ai','Udacity','Udemy','LinkedIn Learning','Cloud',' University','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['AutoML tools','Count']


plt.figure(figsize=(18,8)) 
sns.barplot(x ='AutoML tools' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,300))
plt.show()

# Q41 : What is the primary tool that you use at work or school to analyze data?
やはりまだまだExcelなどのスプレッドシートが分析の主流であり、クラウドベースで行うところは一部のようです。<BR>
TableauなどのBIツールが以外にもクラウドよりも少ないですね。

In [ ]:
df = Survey_JP['Q41'].value_counts(True).reset_index()
df.columns = ['business intelligence','Count']

plt.figure(figsize=(15,8)) 

sns.barplot(x = 'business intelligence', y = 'Count', data = df, alpha=0.7)
 
# Show the plot
plt.xticks(rotation=270)
plt.legend(loc='upper right')
plt.xlabel("business intelligence")
plt.ylabel("Count")
plt.show()

# Q42 : Who/what are your favorite media sources that report on data science topics?
こちらも面白いところですが、YouTubeの後にTwitterが来ています。<BR>
私もTwitterで機械学習系の方をフォローして情報収集をしたりします。

In [ ]:
Cols = ['Q42_Part_1','Q42_Part_2','Q42_Part_3','Q42_Part_4','Q42_Part_5','Q42_Part_6','Q42_Part_7','Q42_Part_8','Q42_Part_9','Q42_Part_10','Q42_Part_11','Q42_OTHER']
Programing = ['Twitter','Email newsletters','Reddit','Kaggle','Course Forums','YouTube','Podcasts','Blogs',
              'Journal Publications','Slack Communities ','None','Other']

df_programing=Survey_JP[Cols]
df_programing.columns = Programing
df=df_programing.count().reset_index(drop=False)
df.columns = ['Media','Count']


plt.figure(figsize=(15,8)) 
sns.barplot(x ='Media' , y = 'Count', data = df.sort_values(by='Count',ascending=False))

plt.xticks(rotation=270)
plt.ylim((0,500))
plt.show()

# Closing Word
長々とお付き合いいただきありがとうございます。<BR>
Kaggleの楽しみ方は、モデリングだけではないと思います。<BR>
今回のように、身近なデータを使ってインサイトを見るのは面白いですね。<BR>
皆さんも良いKaggle Lifeを。<BR>